In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import os
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        #Defining 2 convolutional layers
        self.convlay1 = nn.Conv2d(3, 6, 5, 1)
        self.convlay2 = nn.Conv2d(6, 16, 5, 1)
        #Defining pooling layer
        self.pool = nn.MaxPool2d(2,2)

        self.dropout = nn.Dropout(0.2)

        #Defining 3 fully connected layers
        self.fullyconlay1 = nn.Linear(16*13*13, 100)
        self.fullyconlay2 = nn.Linear(100, 50)
        self.fullyconlay3 = nn.Linear(50, 10) #Final output channels need to be 10

    def forward(self, image_data):
        #First pass
        image_data = F.relu(self.convlay1(image_data)) #Featured maps
        image_data = self.pool(image_data) #Pooled featured maps
        image_data = self.dropout(image_data)
        #Second pass
        image_data = F.relu(self.convlay2(image_data)) #Featured maps
        image_data = self.pool(image_data) #Pooled featured maps
        image_data = self.dropout(image_data)
        #Flatten layer
        image_data = torch.flatten(image_data, 1)
        #Pass through fully connected layers using relu
        image_data = F.relu(self.fullyconlay1(image_data))
        image_data = F.relu(self.fullyconlay2(image_data))
        image_data = self.dropout(image_data)
        return self.fullyconlay3(image_data)

In [4]:
import kagglehub

# Download latest version
dataset_path = kagglehub.dataset_download("karimabdulnabi/fruit-classification10-class")
dataset_path = os.path.join(dataset_path, "MY_data")
# Setup path to data folder
train_dir = os.path.join(dataset_path, "train")
test_dir = os.path.join(dataset_path, "test")
predict_dir = os.path.join(dataset_path, "predict")

train_dir, test_dir


('/root/.cache/kagglehub/datasets/karimabdulnabi/fruit-classification10-class/versions/1/MY_data/train',
 '/root/.cache/kagglehub/datasets/karimabdulnabi/fruit-classification10-class/versions/1/MY_data/test')

In [17]:
from tqdm.auto import tqdm

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer):
    model.train()
    train_loss, train_acc = 0, 0

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module):
    model.eval()
    test_loss, test_acc = 0, 0

    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            test_pred_logits = model(X)

            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5,
          scheduler: torch.optim.lr_scheduler._LRScheduler = None):

    results = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(
            model=model,
            dataloader=train_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer
        )

        test_loss, test_acc = test_step(
            model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn
        )

        # Step the scheduler if it exists
        if scheduler is not None:
            if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                scheduler.step(test_loss)
            else:
                scheduler.step()

        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss if isinstance(train_loss, float) else train_loss.item())
        results["train_acc"].append(train_acc if isinstance(train_acc, float) else train_acc.item())
        results["test_loss"].append(test_loss if isinstance(test_loss, float) else test_loss.item())
        results["test_acc"].append(test_acc if isinstance(test_acc, float) else test_acc.item())

    return results

In [14]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(20),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.RandomResizedCrop(64, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
])

In [15]:
train_data = datasets.ImageFolder(train_dir, transform=transform)
test_data = datasets.ImageFolder(test_dir, transform=transform)

train_data, train_data

(Dataset ImageFolder
     Number of datapoints: 2301
     Root location: /root/.cache/kagglehub/datasets/karimabdulnabi/fruit-classification10-class/versions/1/MY_data/train
     StandardTransform
 Transform: Compose(
                Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
                RandomHorizontalFlip(p=0.5)
                RandomRotation(degrees=[-20.0, 20.0], interpolation=nearest, expand=False, fill=0)
                RandomAffine(degrees=[0.0, 0.0], translate=(0.1, 0.1))
                RandomResizedCrop(size=(64, 64), scale=(0.8, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear, antialias=True)
                ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=(0.8, 1.2), hue=None)
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ),
 Dataset ImageFolder
     Number of datapoints: 2301
     Root location: /root/.cache/kagglehub/datasets/karimabdulnab

In [16]:
import os
BATCH_SIZE = 32  # Instead of 1
train_dataloader = DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    num_workers=2,  # Increased from 1
    shuffle=True
)

test_dataloader = DataLoader(
    dataset=test_data,
    batch_size=BATCH_SIZE,
    num_workers=2,  # Increased from 1
    shuffle=False
)


In [19]:
torch.manual_seed(42)
model = CNN().to(device)
model

NUM_EPOCHS = 50
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.1,
    patience=3,
    min_lr=1e-6
)


from timeit import default_timer as timer
start_time = timer()


results = train(model=model,train_dataloader=train_dataloader,test_dataloader=test_dataloader,optimizer=optimizer,
                        loss_fn=loss_fn,
                        epochs=NUM_EPOCHS, scheduler=scheduler)

end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 2.0612 | train_acc: 0.1839 | test_loss: 1.9035 | test_acc: 0.2188
Epoch: 2 | train_loss: 1.7812 | train_acc: 0.3116 | test_loss: 1.8632 | test_acc: 0.2794
Epoch: 3 | train_loss: 1.6983 | train_acc: 0.3416 | test_loss: 1.7533 | test_acc: 0.3087
Epoch: 4 | train_loss: 1.6425 | train_acc: 0.3725 | test_loss: 1.7209 | test_acc: 0.3419
Epoch: 5 | train_loss: 1.5805 | train_acc: 0.4035 | test_loss: 1.7553 | test_acc: 0.3409


KeyboardInterrupt: 